In [1]:
#pip install pysal
#pip install osmnx
#pip install spreg
#pip install geopandas==0.10.2

In [1]:
from scipy import stats
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import osmnx as ox
sns.set(style="whitegrid")
from pysal.model import spreg
from pysal.lib import weights

import folium as flm
from shapely.geometry import Polygon, MultiPolygon
import branca
from datetime import datetime


## Load data

In [55]:
crime_data = pd.read_csv("data/homicidetypebyprovince.csv")
reg_df = gpd.read_file("data/geo_data/regions_simplified.geojson")
prov_df = gpd.read_file("data/geo_data/provinces_simplified.geojson")


In [24]:
crime_data.shape

(107, 19)

In [25]:
crime_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   prov_name            107 non-null    object
 1   prov_istat_code_num  107 non-null    int64 
 2   prov_acr             106 non-null    object
 3   region_name          107 non-null    object
 4   reg_istat_code       107 non-null    int64 
 5   reg_istat_code_num   107 non-null    int64 
 6   prov_istat_code      107 non-null    int64 
 7   geometry             107 non-null    object
 8   Year                 107 non-null    int64 
 9   region_id            107 non-null    int64 
 10  region_name.1        107 non-null    object
 11  capital_id           107 non-null    object
 12  capital_name         107 non-null    object
 13  type                 107 non-null    object
 14  n_commited_crime     107 non-null    int64 
 15  Year.1               107 non-null    int64 
 16  prov_nam

In [26]:
crime_data

,prov_name,prov_istat_code_num,prov_acr,region_name,reg_istat_code,reg_istat_code_num,prov_istat_code,geometry,Year,region_id,region_name.1,capital_id,capital_name,type,n_commited_crime,Year.1,prov_name.1,Type of Homicide,Number of Homicide
0,Torino,1,TO,Piemonte,1,1,1,"POLYGON ((7.893969937235709 45.58221680371771,...",2017,15,Sicilia,15a,Agrigento,01.0 ATTENTATI,0,2006,Torino,mass murder,0
1,Vercelli,2,VC,Piemonte,1,1,2,"POLYGON ((7.929003239798809 45.74244267625574,...",2017,15,Sicilia,15a,Agrigento,02. 0 STRAGE,1,2006,Genova,mass murder,2
2,Novara,3,NO,Piemonte,1,1,3,POLYGON ((8.420786872922113 45.829807305539404...,2017,15,Sicilia,15a,Agrigento,03.0 OMICIDI VOLONTARI CONSUMATI,2,2006,Milano,mass murder,0
3,Cuneo,4,CN,Piemonte,1,1,4,MULTIPOLYGON (((6.94540332317303 44.4279439457...,2017,15,Sicilia,15a,Agrigento,03.a Omicidi a scopo di furto o rapina,0,2006,Verona,mass murder,0
4,Asti,5,AT,Piemonte,1,1,5,"POLYGON ((7.966847433785602 45.11667127823421,...",2017,15,Sicilia,15a,Agrigento,03.b Omicidio di tipo mafioso,1,2006,Venezia,mass murder,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,Sassari,90,SS,Sardegna,20,20,90,MULTIPOLYGON (((9.465018482973116 40.655838523...,2017,12,Piemonte,12a,Alessandria,13.0 ATTI SESSUALI CON MINORENNE,2,2014,Firenze,mass murder,0
103,Nuoro,91,NU,Sardegna,20,20,91,MULTIPOLYGON (((9.2803660685526 39.91741076929...,2017,12,Piemonte,12a,Alessandria,14.0 CORRUZIONE DI MINORENNE,0,2014,Roma,mass murder,2
104,Cagliari,92,CA,Sardegna,20,20,92,MULTIPOLYGON (((8.848718495176572 39.099237314...,2017,12,Piemonte,12a,Alessandria,15.0 FURTI,2129,2014,Napoli,mass murder,0
105,Oristano,95,OR,Sardegna,20,20,95,MULTIPOLYGON (((8.782004905602278 40.189816014...,2017,12,Piemonte,12a,Alessandria,15.a Furto con strappo,21,2014,Bari,mass murder,0


In [27]:
reg_df = reg_df.to_crs('EPSG:4326')
reg_df['reg_lon'] = reg_df['geometry'].centroid.x
reg_df['reg_lat'] = reg_df['geometry'].centroid.y
reg_df

C:\Users\giuli\AppData\Local\Temp\ipykernel_14160\4086397198.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  reg_df['reg_lon'] = reg_df['geometry'].centroid.x
C:\Users\giuli\AppData\Local\Temp\ipykernel_14160\4086397198.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  reg_df['reg_lat'] = reg_df['geometry'].centroid.y


,reg_name,reg_istat_code_num,reg_istat_code,geometry,reg_lon,reg_lat
0,Piemonte,1,01,"POLYGON ((7.10433 45.46696, 7.11351 45.43406, ...",7.924496,45.060965
1,Valle d'Aosta/Vallée d'Aoste,2,02,"POLYGON ((7.86405 45.91644, 7.80009 45.91712, ...",7.388411,45.728122
2,Lombardia,3,03,"POLYGON ((8.71482 46.09804, 8.72643 46.01797, ...",9.769606,45.620589
3,Trentino-Alto Adige/Südtirol,4,04,"POLYGON ((10.84015 45.83276, 10.88759 45.81165...",11.282438,46.442437
4,Veneto,5,05,"POLYGON ((10.84015 45.83276, 10.62847 45.60206...",11.854295,45.647533
5,Friuli-Venezia Giulia,6,06,"POLYGON ((13.78044 45.61215, 13.80891 45.60674...",13.052323,46.153181
6,Liguria,7,07,"MULTIPOLYGON (((7.71433 44.06156, 7.66303 44.0...",8.698079,44.263332
7,Emilia-Romagna,8,08,"POLYGON ((11.42674 44.95008, 11.30158 44.96236...",11.041628,44.523281
8,Toscana,9,09,"MULTIPOLYGON (((9.68670 44.36594, 9.76542 44.3...",11.125237,43.450333
9,Umbria,10,10,"MULTIPOLYGON (((11.89496 42.83467, 11.92852 42...",12.489537,42.964802


In [28]:
prov_df = prov_df.to_crs('EPSG:4326')
prov_df['prov_lon'] = prov_df['geometry'].centroid.x
prov_df['prov_lat'] = prov_df['geometry'].centroid.y
prov_df

C:\Users\giuli\AppData\Local\Temp\ipykernel_14160\1803136499.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  prov_df['prov_lon'] = prov_df['geometry'].centroid.x
C:\Users\giuli\AppData\Local\Temp\ipykernel_14160\1803136499.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  prov_df['prov_lat'] = prov_df['geometry'].centroid.y


,prov_name,prov_istat_code_num,prov_acr,reg_name,reg_istat_code,reg_istat_code_num,prov_istat_code,geometry,prov_lon,prov_lat
0,Torino,1,TO,Piemonte,01,1,001,"POLYGON ((8.02702 45.40310, 8.03840 45.44143, ...",7.439237,45.144569
1,Vercelli,2,VC,Piemonte,01,1,002,"POLYGON ((7.86405 45.91644, 7.86264 45.79216, ...",8.210807,45.524228
2,Novara,3,NO,Piemonte,01,1,003,"POLYGON ((8.59380 45.82825, 8.56026 45.85184, ...",8.555280,45.564964
3,Cuneo,4,CN,Piemonte,01,1,004,"POLYGON ((7.06535 44.71383, 7.07193 44.68010, ...",7.589371,44.477268
4,Asti,5,AT,Piemonte,01,1,005,"POLYGON ((8.12897 45.12300, 8.09637 45.13174, ...",8.188233,44.879498
...,...,...,...,...,...,...,...,...,...,...
102,Sassari,90,SS,Sardegna,20,20,090,"MULTIPOLYGON (((8.62025 40.31899, 8.64393 40.2...",8.959514,40.746464
103,Nuoro,91,NU,Sardegna,20,20,091,"POLYGON ((9.16558 39.83257, 9.20019 39.82632, ...",9.370500,40.182140
104,Cagliari,92,CA,Sardegna,20,20,092,"POLYGON ((9.44229 39.12694, 9.48639 39.20715, ...",9.106880,39.207028
105,Oristano,95,OR,Sardegna,20,20,095,"POLYGON ((9.16558 39.83257, 9.12881 39.88853, ...",8.716397,39.988005


Merge dataset with crime and the province database.

In [35]:
crime_merged = pd.merge(crime_data, prov_df[['prov_name', 'prov_istat_code_num', 'reg_istat_code_num','prov_lon','prov_lat']], on='prov_name', how='outer')
crime_merged.head(5)
data = crime_merged

In [40]:
# Read OSM data - get administrative boundaries

# get the boundaries of the place (add additional buffer around the query)
boundaries = ox.geocode_to_gdf("Italia", buffer_dist=5000)

# Let's check the boundaries on a map
boundaries.explore()

In [41]:
data["geometry"] = gpd.points_from_xy(data["prov_lon"], data["prov_lat"])
data = gpd.GeoDataFrame(data, crs="epsg:4326")

# Filter data geographically
data = gpd.sjoin(data, boundaries[["geometry"]])
data = data.reset_index(drop=True)

# Check the first rows
data.head()

,prov_name,prov_istat_code_num_x,prov_acr,region_name,reg_istat_code,reg_istat_code_num_x,prov_istat_code,geometry,Year,region_id,...,n_commited_crime,Year.1,prov_name.1,Type of Homicide,Number of Homicide,prov_istat_code_num_y,reg_istat_code_num_y,prov_lon,prov_lat,index_right
0,Torino,1,TO,Piemonte,1,1,1,POINT (7.43924 45.14457),2017,15,...,0,2006,Torino,mass murder,0,1,1,7.439237,45.144569,0
1,Vercelli,2,VC,Piemonte,1,1,2,POINT (8.21081 45.52423),2017,15,...,1,2006,Genova,mass murder,2,2,1,8.210807,45.524228,0
2,Novara,3,NO,Piemonte,1,1,3,POINT (8.55528 45.56496),2017,15,...,2,2006,Milano,mass murder,0,3,1,8.555280,45.564964,0
3,Cuneo,4,CN,Piemonte,1,1,4,POINT (7.58937 44.47727),2017,15,...,0,2006,Verona,mass murder,0,4,1,7.589371,44.477268,0
4,Asti,5,AT,Piemonte,1,1,5,POINT (8.18823 44.87950),2017,15,...,1,2006,Venezia,mass murder,0,5,1,8.188233,44.879498,0


In [56]:
 #data. loc[data["n_commited_crime"]. idxmax()]

In [53]:
data.explore(column="n_commited_crime", cmap="Reds", scheme="quantiles", k=8, tooltip=["n_commited_crime"], vmax=15)


C:\Users\giuli\anaconda3\lib\site-packages\mapclassify\classifiers.py:255: UserWarning: Not enough unique values in array to form 8 classes. Setting k to 6.
  warnings.warn(
